# Using Simplicit's Easy API To Simulate Example Mesh
[Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/) is a mesh-free, representation-agnostic way to simulation elastic deformations. 

Here's a simple way to use the simplicit's code base. We can create a simple object, train it, simulate it and visualize all in a very few lines of code via our `easy_api`.

In [ ]:
import copy, math, os, sys, logging, threading
from typing import List, Tuple
from pathlib import Path

import numpy as np
import torch
import kaolin as kal

from IPython.display import display
from ipywidgets import Button, HBox, VBox

#local logger, prints at info or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

#logger used in the api code
logging.getLogger('kaolin.physics').setLevel(logging.INFO) # Prints everything at debug level or above

sys.path.append(str(Path("..")))
from tutorial_common import COMMON_DATA_DIR

def sample_mesh_path(fname):
    return os.path.join(COMMON_DATA_DIR, 'meshes', fname)

def print_tensor(t, name='', **kwargs):
    print(kal.utils.testing.tensor_info(t, name=name, **kwargs))

## Loading Geometry
Simplicits works with any geometry: meshes, pointclouds, SDFs, Gaussian splats, and more. For this example, we will use a mesh:

In [ ]:
# Import and triangulate to enable rasterization; move to GPU
mesh = kal.io.import_mesh(sample_mesh_path('fox.obj'), triangulate=True).cuda()
mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices.unsqueeze(0), normalize=True).squeeze(0) 
orig_vertices = mesh.vertices.clone()  # Also save original undeformed vertices
print(mesh)

## Sample Geometry
To enable simulation we need point samples within the object's volume, and physical material parameters per point. Lets set this up.

In [ ]:
# Physics material parameters
soft_youngs_modulus = 1e5
poisson_ratio = 0.45
rho = 500  # kg/m^3
approx_volume = 0.5  # m^3

# Points sampled over the object's bounding box
num_samples = 1000000
uniform_pts = torch.rand(num_samples, 3, device='cuda') * (orig_vertices.max(dim=0).values - orig_vertices.min(dim=0).values) + orig_vertices.min(dim=0).values
boolean_signs = kal.ops.mesh.check_sign(mesh.vertices.unsqueeze(0), mesh.faces, uniform_pts.unsqueeze(0), hash_resolution=512)

# use pts within the object
pts = uniform_pts[boolean_signs.squeeze()]
yms = torch.full((pts.shape[0],), soft_youngs_modulus, device="cuda")
prs = torch.full((pts.shape[0],), poisson_ratio, device="cuda")
rhos = torch.full((pts.shape[0],), rho, device="cuda")

import k3d
plot = k3d.plot()
plot += k3d.points(pts.cpu().detach().numpy(), point_size=0.01)
plot.display()

## Create and Train a SimplicitsObject
We encapsulate everything Simplicits method needs to know about the simulated object in a `SimplicitsObject` instance. Once the object is created, we need to run training to learn reduced degrees of freedom our simulator can use. 

**This will take a couple of minutes.** Please be patient.

In [ ]:
# Initialize and train a Simpicits object to enable simulation
sim_obj = kal.physics.simplicits.SimplicitsObject(pts, yms, prs, rhos, torch.tensor([approx_volume], dtype=torch.float32, device="cuda"), num_handles=5)
print('Training simplicits object. This will take 2-3min. ')
sim_obj.train(num_steps=10000)  # TODO: with next patch add log_interval=1000
print('Object ready to simulate.')

# sim_obj.load_model('/tmp/test_easy_api.pt') # if you saved previously trained object, you can load it instead

In [ ]:
# Optionally, you can save/load this pre-trained object
# sim_obj.save_model('fox_mesh_model_10k_steps.pt')

## Create a Scene
Now we are ready to set up all the forces in the scene to simulated as well as simulation settings. For example, here we will add gravity and a floor plane.

In [ ]:
scene = kal.physics.simplicits.SimplicitsScene() # default empty scene
#Convergence might not be guaranteed with few newton iterations, but runs very fast
scene.max_newton_steps = 3 

 The same `SimplicitsObject` can be added to multiple scene. Let's add it to our scene. Not we can reference it within the scene using `obj_idx`.

In [ ]:
obj_idx = scene.add_object(sim_obj)

Lets set gravity and floor forces on the scene

In [ ]:
scene.set_scene_gravity(acc_gravity=torch.tensor([0, 9.8, 0]))
scene.set_scene_floor(floor_height=-0.8, floor_axis=1, floor_penalty=1000)

We can play around with the material parameters of the object, indicated via `object_idx`

In [ ]:
scene.set_object_materials(obj_idx, yms=torch.tensor(1e4, device='cuda', dtype=torch.float))

## Set Up Rendering
Let's set up rendering of our mesh so we can view it in a notebook.

In [ ]:
resolution = 512
camera = kal.render.easy_render.default_camera(resolution).cuda()

light_direction = kal.render.lighting.sg_direction_from_azimuth_elevation(1., 1.)
lighting = kal.render.lighting.SgLightingParameters(amplitude=3., sharpness=5., direction=light_direction).cuda()
    
def render(in_cam):
    # render
    active_pass=kal.render.easy_render.RenderPass.render
    render_res = kal.render.easy_render.render_mesh(in_cam, mesh, lighting=lighting)

    # create white background
    img = render_res[active_pass]
    background_mask = (render_res[kal.render.easy_render.RenderPass.face_idx] < 0).bool()
    img2 = torch.clamp(img, 0, 1)[0]
    img2[background_mask[0]] = 1
    final = (img2 * 255.).to(torch.uint8)
    return {"img":final, "face_idx": render_res[kal.render.easy_render.RenderPass.face_idx].squeeze(0).unsqueeze(-1)}

# faster low-res render during mouse motion
def fast_render(in_cam, factor=8):
    lowres_cam = copy.deepcopy(in_cam)
    lowres_cam.width = in_cam.width // factor
    lowres_cam.height = in_cam.height // factor
    return render(lowres_cam)

## That's it! Let's Run and View or Physics Simulation
All we need to do now is run simulation and display the object using Kaolin's in-notebook visualizer.

In [ ]:
# Reset mesh to its rest state
mesh.vertices = orig_vertices

fox_verts = []

global sim_thread_open, sim_thread
sim_thread_open = False
sim_thread = None

def run_sim():
    scene.reset()  # reset internal simultion state

    for s in range(100):
        with visualizer.out:
            scene.run_sim_step()
            print(".", end="")
        mesh.vertices = scene.get_object_deformed_pts(obj_idx, orig_vertices).squeeze()
        if s%10 == 0:
            fox_verts.append(mesh.vertices.clone().detach())
        visualizer.render_update()

def start_simulation(b):
    global sim_thread_open, sim_thread
    with visualizer.out:
        if(sim_thread_open):
            sim_thread.join()
            sim_thread_open = False
        sim_thread_open = True
        sim_thread = threading.Thread(target=run_sim, daemon=True)
        sim_thread.start()

scene.reset_object(obj_idx)
button = Button(description='Run Sim')
button.on_click(start_simulation)
visualizer = kal.visualize.IpyTurntableVisualizer(
    resolution, resolution, copy.deepcopy(camera), render, fast_render=fast_render,
    max_fps=24, world_up_axis=1)
visualizer.render_update()  # render first frame
display(HBox([visualizer.canvas, button]), visualizer.out)